
# TensorFlow Training
Train multiple frozen-backbone classifiers on the preprocessed PlantVillage dataset. No augmentation here; dataset is already balanced/augmented.


In [5]:

import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5

PROJECT_ROOT = Path(".").resolve()
PREPROCESSED_DIR = PROJECT_ROOT / "preprocessed"
RESULTS_DIR = PROJECT_ROOT / "results"
MODELS_DIR = PROJECT_ROOT / "models"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)


## 1) Data loaders (PNG + CSV labels)

In [6]:

def load_split(split_name):
    csv_path = PREPROCESSED_DIR / f"{split_name}_labels.csv"
    df = pd.read_csv(csv_path)
    filepaths = str(PREPROCESSED_DIR / split_name) + os.sep + df["filename"].astype(str)
    labels = df["label"].astype(np.int32)
    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    def _load(path, label):
        image = tf.io.read_file(path)
        image = tf.io.decode_png(image, channels=3)
        image = tf.image.resize(image, IMAGE_SIZE)
        image = tf.cast(image, tf.float32) / 255.0
        return image, tf.cast(label, tf.float32)

    return ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


train_ds = load_split("train")
val_ds = load_split("val")
test_ds = load_split("test")


## 2) Model factory

In [7]:

def create_binary_classifier(base_model_fn, input_shape=(224, 224, 3)):
    base_model = base_model_fn(input_shape=input_shape, include_top=False, weights="imagenet")
    base_model.trainable = False
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inputs=base_model.input, outputs=output)
    return model


def get_model_builder(model_key):
    key = model_key.lower()
    if key == "mobilenet_v2":
        return lambda: create_binary_classifier(tf.keras.applications.MobileNetV2)
    if key == "efficientnet_b0":
        return lambda: create_binary_classifier(tf.keras.applications.EfficientNetB0)
    if key == "resnet50":
        return lambda: create_binary_classifier(tf.keras.applications.ResNet50)
    if key == "vgg16":
        return lambda: create_binary_classifier(tf.keras.applications.VGG16)
    if key == "densenet121":
        return lambda: create_binary_classifier(tf.keras.applications.DenseNet121)
    if key == "inception_v3":
        return lambda: create_binary_classifier(tf.keras.applications.InceptionV3)
    raise ValueError(f"Unknown model key: {model_key}")


MODELS_TO_TRAIN = [
#    "mobilenet_v2", 
#    "efficientnet_b0",
    "resnet50",
                   ]


## 3) Training + evaluation loop

In [ ]:

def compile_model(model, lr=1e-3):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name="accuracy"),
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall"),
            tf.keras.metrics.AUC(name="auc"),
        ],
    )


def evaluate_metrics(model, test_ds):
    y_true = []
    y_prob = []
    for images, labels in test_ds:
        probs = model.predict(images, verbose=0).flatten()
        y_prob.extend(probs)
        y_true.extend(labels.numpy())
    y_true = np.array(y_true)
    y_prob = np.array(y_prob)
    y_pred = (y_prob > 0.5).astype(int)
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred)),
        "recall": float(recall_score(y_true, y_pred)),
        "f1": float(f1_score(y_true, y_pred)),
        "auc": float(roc_auc_score(y_true, y_prob)),
    }


all_rows = []
for model_key in MODELS_TO_TRAIN:
    print(f" === Training {model_key} ===")
    builder = get_model_builder(model_key)
    model = builder()
    compile_model(model)

    ckpt_path = MODELS_DIR / f"{model_key}.h5"
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=str(ckpt_path), monitor="val_accuracy", save_best_only=True, verbose=1),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1, min_lr=1e-6, verbose=1),
    ]

    t0 = time.time()
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks, verbose=1)
    train_time_sec = time.time() - t0

    # metrics = evaluate_metrics(model, test_ds)
    # row = {"framework": "tensorflow", "model": model_key, **metrics, "train_time_sec": train_time_sec, "train_time_min": train_time_sec / 60.0, "checkpoint": str(ckpt_path)}
    # all_rows.append(row)
    # print(row)
    # ...existing code...
    metrics = evaluate_metrics(model, test_ds)
    row = {"framework": "tensorflow", "model": model_key, **metrics, "train_time_sec": train_time_sec, "train_time_min": train_time_sec / 60.0, "checkpoint": str(ckpt_path)}
    all_rows.append(row)
    print(row)

    # --- new: merge/update single-row into results CSV without overwriting other entries ---
    csv_path = RESULTS_DIR / "tf_metrics.csv"
    if csv_path.exists():
        existing_df = pd.read_csv(csv_path)
        # remove any previous entry for same (framework, model)
        mask = ~((existing_df["framework"] == row["framework"]) & (existing_df["model"] == row["model"]))
        existing_df = existing_df[mask]
        combined_df = pd.concat([existing_df, pd.DataFrame([row])], ignore_index=True)
    else:
        combined_df = pd.DataFrame([row])
    combined_df.to_csv(csv_path, index=False)
    print(f"Saved/updated TensorFlow metrics -> {csv_path}")

    

# tf_df = pd.DataFrame(all_rows)
# tf_metrics_path = RESULTS_DIR / "tf_metrics.csv"
# tf_df.to_csv(tf_metrics_path, index=False)
# print(f"Saved TensorFlow metrics -> {tf_metrics_path}")
# tf_df


 === Training resnet50 ===
Epoch 1/5
1510/2186 ━━━━━━━━━━━━━━━━━━━━ 1:44 154ms/step - accuracy: 0.9602 - auc: 0.2584 - loss: 0.0783 - precision: 0.1095 - recall: 0.1000

In [ ]:
# ...existing code...
    
# ...existing code...